In [1]:
import pandas as pd
df = pd.read_pickle("udemy_reviews.pkl")
df['tag']=df['rating'].apply(lambda x: 'pos' if x > 4 else 'neg')
df.sample(10)

,id,course,rating,comment,user,tag
31980,71752316,611804,4.0,excelente,Josué David Cutire Condori,neg
13100,47206248,976214,5.0,bueno,Kemy Joan Hinostroza Jorge,pos
63013,56588464,749262,5.0,El profesor comprte trucos que agilizan la tar...,Roxana Paola Crisanto Silupú,pos
40518,61854062,2221894,4.0,Increíble el curso...nos encanto aprender a ha...,Cesar Duran,neg
53153,83398846,1159626,5.0,Excelente curso,Hortensia Artidiello,pos
54530,45545637,1690922,5.0,"Bastante bien explicado, casos teóricos entend...",Omar Avila,pos
54414,34271518,1902716,3.0,"Regular, el sonido no es bueno y el close capt...",Héctor García Elejalde,neg
36932,6325622,899474,5.0,explica lo necesario y de forma clara,Graciela,pos
24981,76419762,3446356,3.5,"no me cuesta trabajo entender, pero sin duda (...",Ivan Vladimir Vera Amieva,neg
29604,88436054,1512696,4.0,Si !!!,Roberto Adasme,neg


In [2]:
# cantidad de cursos distintos sobre los que se analizan las reviews
len(pd.unique(df['course']))

5065

In [3]:
# cantidad de reviews
len(df)

174141

In [4]:
# Cantidad de reviews positivas y negativas
df['tag'].value_counts()

pos    126660
neg     47481
Name: tag, dtype: int64

In [5]:
from nltk import word_tokenize, sent_tokenize
import re

In [6]:
texts_by_sentences = []
for rev in df['comment']:
    rev = rev.lower()
    rev = re.sub('<.*?>',' ',rev)
    sentences = sent_tokenize(rev)
    for sent in sentences:
        texts_by_sentences.append([word for word in word_tokenize(sent) if word.isalpha()])

In [7]:
texts_by_sentences[:3]

[['como',
  'introducción',
  'esta',
  'bien',
  'pero',
  'deberían',
  'de',
  'hacerlo',
  'mas',
  'dirigido',
  'a',
  'rpa',
  'con',
  'fotos',
  'de',
  'rpas'],
 ['los',
  'conocimientos',
  'son',
  'claros',
  'y',
  'concisos',
  'para',
  'lo',
  'requerido'],
 ['hermoso']]

In [8]:
print(f"el corpus tiene {len(texts_by_sentences)} oraciones y {sum([len(x) for x in texts_by_sentences])} palabras")

el corpus tiene 242744 oraciones y 2619372 palabras


In [9]:
from gensim.models import Phrases
from gensim.models.phrases import Phraser

collocations = Phrases(sentences=texts_by_sentences, 
                       min_count=3, threshold=0.4, scoring='npmi')
to_collocations = Phraser(collocations)
data_ngrams = to_collocations[texts_by_sentences]

In [10]:
len(collocations.export_phrases())

# algunas collocations:
#  super_recomendado
#  página_oficial
#  buen_ritmo
#  vida_profesional

6047

In [11]:
data_ngrams

In [12]:
from gensim.models.phrases import Phrases, Phraser

df_collocations = pd.DataFrame([x for x in collocations.find_phrases(texts_by_sentences).items()],
                               columns=["bigram","score"])
df_collocations.shape

(5820, 2)

In [13]:
df_collocations = df_collocations.drop_duplicates().sort_values(by='score',ascending=False)
df_collocations.head(10)

,bigram,score
3173,touch_fwconsole,1.000000
5355,dalla_benetta,1.000000
5733,capio_biogás,1.000000
2024,packet_tracer,0.995215
3479,qi_gong,0.992972
2943,allen_bradley,0.990725
5723,blue_phoenix,0.989594
3781,lazy_load,0.988130
5497,pixel_art,0.988020
3167,vielen_dank,0.983338


In [14]:
from gensim.models import Word2Vec

w2v_model = Word2Vec(sentences=data_ngrams, 
                 workers=4, vector_size=20, min_count=10, window=10,
                 sample=1e-3, negative=5, sg=1)


In [15]:
w2v_model.wv["excelente"]

array([-0.18754354, -0.5596569 ,  1.1118038 ,  0.659424  ,  0.3268008 ,
        0.57661074,  0.33021903, -0.48809192,  0.35598928, -0.19273773,
        1.019242  , -0.55781734,  0.28589052,  0.3287943 , -0.3105603 ,
        0.4214851 ,  0.30508074,  0.15681122, -0.60300416, -0.36157164],
      dtype=float32)

In [16]:
w2v_model.wv.similarity("comunicación","profesor")

0.53571326

In [17]:
w2v_model.wv.most_similar("excelente")

[('muy_buena', 0.9140053391456604),
 ('exelente', 0.9043107628822327),
 ('excelete', 0.8969972133636475),
 ('buenisima', 0.8716444969177246),
 ('gran', 0.8660933971405029),
 ('impecable', 0.8634282350540161),
 ('brillante', 0.8402697443962097),
 ('perfecta', 0.8369444608688354),
 ('extraordinaria', 0.8357675671577454),
 ('excelnte', 0.8335145711898804)]

In [18]:
w2v_model.wv.most_similar(positive="clase",topn=10)

[('palabra', 0.8982869982719421),
 ('receta', 0.8909982442855835),
 ('chica', 0.8711385726928711),
 ('velocidad', 0.8491659164428711),
 ('lección', 0.8413988351821899),
 ('pizarra', 0.8368756175041199),
 ('imagen', 0.8298501372337341),
 ('gráfica', 0.8268459439277649),
 ('secuencia', 0.8177201747894287),
 ('utilizada', 0.8173375129699707)]

In [19]:
from gensim.corpora import Dictionary

dictionary = Dictionary(texts_by_sentences)


dictionary.filter_extremes(no_below=5, no_above=0.3)#, keep_n=10000)
#dictionary.filter_n_most_frequent(10000)
len(dictionary)

12451

In [20]:
list(dictionary.iteritems())[:15]

[(0, 'a'),
 (1, 'bien'),
 (2, 'como'),
 (3, 'con'),
 (4, 'de'),
 (5, 'deberían'),
 (6, 'dirigido'),
 (7, 'esta'),
 (8, 'fotos'),
 (9, 'hacerlo'),
 (10, 'introducción'),
 (11, 'mas'),
 (12, 'pero'),
 (13, 'rpa'),
 (14, 'claros')]

In [21]:
from gensim import models

corpus = [dictionary.doc2bow(line) for line in texts_by_sentences]
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

In [22]:
n_topics = 40
lsa_tfidf = models.LsiModel(corpus_tfidf,
                            id2word=dictionary,
                            num_topics=n_topics)

In [23]:
vect_profesor = lsa_tfidf[dictionary.doc2bow(["profesor"])]
vect_profesor.sort(key = lambda x: -x[1]) 
vect_profesor[0:10]

[(36, 0.5518219619851673),
 (37, 0.19210459995191465),
 (38, 0.16428657019436574),
 (16, 0.07537895666652157),
 (25, 0.06478142326062918),
 (31, 0.05554759137057093),
 (29, 0.043500191920834715),
 (1, 0.029091825088525987),
 (7, 0.028985011641871315),
 (15, 0.02881222230284073)]

In [24]:
lsa_tfidf.show_topic(36,topn=15)

[('profesor', 0.5518219619851673),
 ('bien', -0.24706628631021346),
 ('bastante', -0.2365234645765334),
 ('explica', 0.2362330935660254),
 ('explicado', 0.23553323057882974),
 ('momento', -0.2287199711216232),
 ('por', -0.22715386251923989),
 ('esta', 0.20455794510768885),
 ('ahora', 0.1899588700077436),
 ('contenido', -0.16609043949615473),
 ('paso', 0.14776679220565853),
 ('un', 0.13503045023388835),
 ('hasta', 0.11997611816091926),
 ('está', -0.11368730959557906),
 ('básico', -0.10744885750105052)]

In [25]:
vect_explicaciones = lsa_tfidf[dictionary.doc2bow(["explicaciones"])]
vect_explicaciones.sort(key = lambda x: -x[1]) 
vect_explicaciones[0:10]

[(20, 0.13510972833141863),
 (35, 0.12923618787816188),
 (18, 0.12675323191951351),
 (16, 0.08448674934624853),
 (22, 0.0665707686889756),
 (23, 0.06203758284081345),
 (19, 0.040755043993810226),
 (15, 0.04050470451748275),
 (12, 0.032406605187537324),
 (24, 0.03158212414534204)]

In [26]:
lsa_tfidf.show_topic(35,topn=15)

[('por', 0.411266125418683),
 ('ha', 0.3618961553902305),
 ('los', -0.28662540068811854),
 ('mucho', -0.2621821217681002),
 ('sido', 0.2288161033210032),
 ('paso', 0.21518938415914565),
 ('contenido', -0.18219531836548195),
 ('hasta', -0.1676879608946621),
 ('gusto', -0.14898267431627302),
 ('conceptos', -0.14618734057051913),
 ('a', 0.14188394038074173),
 ('explicaciones', 0.1292361878781618),
 ('ahora', -0.12910962322756012),
 ('fue', -0.12273486865019946),
 ('momento', 0.11942032034798539)]

In [27]:
#Las palabras que tienen más presencia en cada topic set
for k in range(40):
    x = lsa_tfidf.show_topic(k,topn=5)
    print(k,x[0][0],x[1][0],x[2][0],x[3][0],x[4][0])

0 excelente curso muy bien explicado
1 muy bien excelente explicado buen
2 gracias muchas bien explicado muy
3 si buena bien explicado elección
4 buen si bien curso explicado
5 si buena elección buen muy
6 buen bueno buena curso el
7 bueno muy si que de
8 claro interesante muy elección bueno
9 hasta momento ahora elección explicación
10 interesante claro todo hasta curso
11 elección claro explicación hasta buena
12 recomiendo lo curso elección buen
13 recomiendo lo me curso buen
14 me mucho es curso recomiendo
15 recomendado es muy explicado explica
16 explicación recomendado explicado explica recomiendo
17 explica explicado bien explicación elección
18 recomendable completo muy claro todo
19 muchas gracias es un me
20 es un muchas curso me
21 todo recomendable el contenido momento
22 recomendable completo el ahora contenido
23 fue eleccion elección una esta
24 todo completo contenido esta el
25 todo esta fue eleccion completo
26 esta de fácil entender explicaciones
27 explicaciones la